In [175]:
from pymongo import MongoClient
import pprint
import pandas as pd

client = MongoClient('ec2-54-173-229-9.compute-1.amazonaws.com:27026')
db = client.gdelt
events = db.events

In [176]:
result = events.count()
print(result)

1463717


In [177]:
import time
t0 = time.time()
d = events.aggregate([{
   "$project": {
       "Day": "$Day",
       "Actor2CountryCode": "$Actor2CountryCode",
       "Actor1CountryCode": "$Actor1CountryCode",
       "GoldsteinScale": "$GoldsteinScale",
       "latitude" : "$Actor1Geo_Lat",
       "longitude" : "$Actor1Geo_Long",
       "typeAvis":
           {"$cond": [ { "$gte": [ "$GoldsteinScale", 0 ] }, "pos", "neg" ]}}},{
   "$match": {
       "Actor2CountryCode": "FRA",
       "Day":{"$lt": 20170522, "$gte": 20170421}}},{
   "$group":{
           "_id":{"dd":"$Day", "cc":"$Actor1CountryCode", "typeAvis":"$typeAvis"},
           "lat":{"$avg": "$latitude"},
           "long":{"$avg": "$longitude"},
           "avgGS":{"$avg": "$GoldsteinScale"}, "minGS":{"$min": "$GoldsteinScale"},
           "maxGS":{"$max": "$GoldsteinScale"}, "numGS":{"$sum":1},
           "stdDevGS": {"$stdDevPop": "$GoldsteinScale" }}}, {
   "$sort": { "Day": 1, "numGS": -1 } }])
t1 = time.time()
print("TIME :", t1-t0)

TIME : 1.607672929763794


In [178]:
df = pd.DataFrame(list(d))
df1 = pd.DataFrame(df['_id'].tolist())
df[['Country_Code','Day','Avis']] = df1
del df['_id']
df['Date'] = pd.to_datetime(df['Day'], format='%Y%m%d')
del df['Day']
df = df.sort_values(by='Date')
df = df.dropna()

In [179]:
countries = pd.read_excel('countries.xlsx')

In [180]:
for cc,lat,long in zip(countries['Code'],countries['lat'],countries['long']):
    df.loc[df['Country_Code'] == cc,'lat'] = lat
    df.loc[df['Country_Code'] == cc,'long'] = long

In [181]:
df.to_csv('data.csv',sep=';')

In [200]:
import folium

In [204]:
maps = {}
for date in df['Date'].unique():
    map_1 = folium.Map(location=[30, 0], width=1000, height=500, zoom_start=2,)
    df7 = df[df['Date'] == date]
    for i, j, numGS, avis, cc in zip(df7['lat'], df7['long'], df7['numGS'], df7['Avis'], df7['Country_Code']):
        if (str(avis) == 'pos'):
            folium.CircleMarker([i, j], radius=10, popup='Country : ' +
                                str(cc) + ' | ' + 'numGS : ' + str(numGS)).add_to(map_1)
        else:
            folium.CircleMarker([i, j], radius=5, popup='Country : ' + str(cc) + ' | ' + 'numGS : ' + str(numGS), color='red').add_to(map_1)
    maps[str(date).split('T')[0]] = map_1